In [1]:
pwd

'C:\\Users\\vigne_u98gpxi\\Desktop\\Lavviggy\\Lav\\HuntJob\\Income\\DE_CaseStudy_Dataset\\Sales_Data'

In [2]:
import sys
print(sys.version)
print(sys.exec_prefix)

3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
C:\Users\vigne_u98gpxi\anaconda3


In [3]:
import pyspark
from pyspark.sql import SparkSession
print(pyspark.__version__)

3.3.0


In [59]:
from pyspark.sql.functions import avg, round, col,sum,count,split,weekofyear,struct,regexp_extract,udf

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import avg, round, col,sum,count,split,weekofyear,struct,regexp_extract
from pyspark.sql.functions import *
from pyspark.sql import DataFrame
from functools import reduce
from pyspark.sql.functions import from_unixtime,to_date, unix_timestamp
#import org.apache.spark.sql.functions.col
#import org.apache.spark.sql.types.IntegerType

def create_spark_session(app_name: str) -> SparkSession:
    """ Create a spark session.
    """
    ss = SparkSession.builder.appName(app_name).config("spark.jars", "mysql-connector-j-8.3.0.jar").getOrCreate()#.master('local').appName(app_name).getOrCreate()
    return ss


def read_in_data(sc: SparkSession, file: str):
    """ Return a spark DataFrame of the excel file <file>.
    """
    return sc.read.csv(file, header='true', sep=',', inferSchema=True)


def unionAll(*dfs):
    """ Return a spark DataFrame combining all the dfs.
    """
    return reduce(DataFrame.unionAll, dfs) 


replacements = {
    "int": 0,
    "double": 0.0,
    "float": 0.0,
    "string": ""
}

def missing_values_imputation(df):
    """ Return a spark DataFrame with missing values computes.
    """
    for col_name, data_type in df.dtypes:
            replacement_value = replacements.get(data_type.lower(), None)
            if replacement_value is not None:
                df = df.withColumn(col_name, when(col(col_name).isNull(), replacement_value).otherwise(col(col_name)))
    return df
    

       

# # def output_result(df: DataFrame, output_location: str, output_folder: str) -> None:
# #     """ Save the DataFrame <df> as a csv file in the location specified by
# #     <output_location>.
# #     """
# #     df.coalesce(1).write.csv(path=output_location + output_folder,
#                              mode='append', header=True)

   
    

if __name__ == '__main__':

    # create a spark session
    spark = create_spark_session("datapipeline")

    #### EXTRACT ####
    
    ####  READ BOOKS DATA ####
    
    # read in the books_data.csv as a spark DataFrame
    books_df = read_in_data(spark, 'books_data.csv')

    # read in the Books_ratingas a spark DataFrame
    books_ratings_df = read_in_data(spark, 'Books_rating.csv')
    
    
    ####  READ SALES DATA ####
    # read in the Sales_2019.csv's a spark DataFrame
    
    sales_jan_df= read_in_data(spark, 'Sales_January_2019.csv')
    sales_feb_df= read_in_data(spark, 'Sales_February_2019.csv')
    sales_mar_df= read_in_data(spark, 'Sales_March_2019.csv')
    sales_april_df= read_in_data(spark, 'Sales_April_2019.csv')
    sales_may_df= read_in_data(spark, 'Sales_May_2019.csv')
    sales_june_df= read_in_data(spark, 'Sales_June_2019.csv')
    sales_july_df= read_in_data(spark, 'Sales_July_2019.csv')
    sales_aug_df= read_in_data(spark, 'Sales_August_2019.csv')
    sales_sep_df= read_in_data(spark, 'Sales_September_2019.csv')
    sales_oct_df= read_in_data(spark, 'Sales_October_2019.csv')
    sales_nov_df= read_in_data(spark, 'Sales_November_2019.csv')
    sales_dec_df= read_in_data(spark, 'Sales_December_2019.csv')
    
    
    ####  LIQUOR SALES DATA ####
    # read in the liquor.csv's a spark DataFrame
    
    liquor_sales_df= read_in_data(spark, 'Liquor_Sales.csv')
    
    #### TRANSFORM ####
    def extract_date_columns_liquor(df,date_col):
        df=df.withColumn('year', split(liquor_sales_df[date_col], '/').getItem(2)) \
       .withColumn('month', split(liquor_sales_df[date_col], '/').getItem(0)) 
        df=df.withColumn(date_col, to_date(date_col, "MM/dd/yyyy"))
        df=df.withColumn("week", weekofyear(date_col))
        return df
        
        
#     liquor_date_df=liquor_sales_df.withColumn('year', split(liquor_sales_df['Date'], '/').getItem(2)) \
#    .withColumn('month', split(liquor_sales_df['Date'], '/').getItem(0)) 
#     liquor_date_df=liquor_date_df.withColumn("Date", to_date("Date", "MM/dd/yyyy"))
#     liquor_date_df=liquor_date_df.withColumn("week", weekofyear("Date"))
      liquor_date_df=extract_date_columns_liquor(liquor_sales_df,"Date")
    
    
    def get_lat_long(df,point_col):
         df=df.withColumn("geometry_cleaned", regexp_replace(point_col, "POINT \\(", ""))\
        .withColumn("geometry_cleaned", regexp_replace("geometry_cleaned", "\\)", ""))\
        .withColumn("lat", split(col("geometry_cleaned"), " ")[1].cast("float")) \
        .withColumn("lon", split(col("geometry_cleaned"), " ")[0].cast("float"))\
        .drop("geometry_cleaned")
        return df   
            
    liquor_date_df= get_lat_long(liquor_date_df,"Store Location")    
    liquor_data_mart_df=missing_values_imputation(liquor_date_df)

     
    

     
#     liquor_date_df=liquor_date_df.withColumn("geometry_cleaned", regexp_replace("Store Location", "POINT \\(", ""))\
# .withColumn("geometry_cleaned", regexp_replace("geometry_cleaned", "\\)", ""))\
# .withColumn("lat", split(col("geometry_cleaned"), " ")[1].cast("float")) \
# .withColumn("lon", split(col("geometry_cleaned"), " ")[0].cast("float"))\
# .drop("geometry_cleaned")
    #liquor_date_df=missing_values_imputation(liquor_date_df)
 
    

    #### TRANSFORM ####


    books_ratings_df=books_ratings_df.withColumn('Price',col('Price').cast("integer"))
    books_ratings_df = books_ratings_df.withColumn("review/time", from_unixtime("review/time"))
  
    # Joing the book data and books_ratings data
    books_df_joined=books_df.join(books_ratings_df,books_df.Title ==  books_ratings_df.Title,"inner").drop(books_ratings_df.Title)
    books_df_joined=missing_values_imputation(books_df_joined)

    
    
    ####### Electronics transformation #######
  
    
    df_sales=unionAll(*[sales_jan_df, sales_feb_df,sales_mar_df,sales_april_df,sales_may_df,sales_june_df
           ,sales_july_df,sales_aug_df,sales_sep_df,sales_oct_df,sales_nov_df,sales_dec_df])
    
    ### SPLIT ADDRESS ###
    
    def split_address(df,address_col):
        df = df.withColumn("city", split(df[address_col],",").getItem(1))\
        .withColumn("state_code", split(df[address_col],",").getItem(2))
        df=df.withColumn("state", split(df["state_code"], " ").getItem(1))
        df = df.drop("state_code")
        return df
   
    df_sales=df_sales.withColumn("Order Date", to_date(unix_timestamp("Order Date", "MM/dd/yy HH:mm").cast("timestamp")))
    df_sales_split=split_address(df_sales,"Purchase Address")
    
#     split_df = df_sales.withColumn("city", split(df_sales["Purchase Address"],",").getItem(1))\
# .withColumn("state_code", split(df_sales["Purchase Address"],",").getItem(2))
#     df_sales_split=split_df.withColumn("state", split(split_df["state_code"], " ").getItem(1))
#     df_sales_split = df_sales_split.drop("state_code")
    
    ### date operations ###
    def extract_date_columns_electronics(df,date_col):
        df=df.withColumn('year', split(df[date_col], '-').getItem(0)) \
       .withColumn('month', split(df[date_col], '-').getItem(1)) 
        df = df.withColumn("orderDateOnly", to_date(date_col))
        df=df.withColumn("week", weekofyear(date_col"))
        return df
    df_date=extract_date_columns_electronics(df_sales_split,"Order Date")
    
#     df_date=df_sales_split.withColumn('year', split(df_sales_split['Order Date'], '-').getItem(0)) \
#        .withColumn('month', split(df_sales_split['Order Date'], '-').getItem(1)) 
#     df_date = df_date.withColumn("orderDateOnly", to_date("Order Date"))
#     df_date=df_date.withColumn("week", weekofyear("Order Date"))
    category_mapping = {
    "1": "Smartphone",
    "2": "Charging Cable",
    "3": "Headphones",
    "4": "Monitor",
    "5": "Batteries",
    "6": "Laptop",
    "7": "TV",
    "8": "Dryer",
    "9": "Washing Machine",
    "10": "others"
}
    df_date=df_date.withColumn("category",
                               when(col("product").contains("Batteries"), "Batteries")
                               .when(col("product").contains("Charging Cable"), "Charging Cable")
                               .when(col("product").contains("Phone"), "Smartphone")
                               .when(col("product").contains("Headphones"), "Headphones")
                               .when(col("product").contains("Laptop"), "Laptop")
                               .when(col("product").contains("TV"), "TV")
                               .when(col("product").contains("Dryer"), "Dryer")
                               .when(col("product").contains("Washing Machine"), "Washing Machine")
                               .otherwise("Others"))
    electronics_data_mart=missing_values_imputation(df_date)
  
    #### LOAD ####

    #OUTPUT_LOCATION = '/output'

    # save the above results as csv files in the bucket
#     output_result(sum_price_by_title, OUTPUT_LOCATION, 'Title')
#     output_result(count_price_by_title, OUTPUT_LOCATION, 'Title')
  



In [29]:


def extract_date_columns_liquor(df,date_col):
    df=df.withColumn('year', split(liquor_sales_df[date_col], '/').getItem(2)) \
   .withColumn('month', split(liquor_sales_df[date_col], '/').getItem(0)) 
    df=df.withColumn(date_col, to_date(date_col, "MM/dd/yyyy"))
    df=df.withColumn("week", weekofyear(date_col))
    return df

In [32]:
extract_date_columns_liquor(liquor_sales_df,"Date")

+-------------------+----------+------------+--------------------+--------------------+---------------+--------+--------------------+-------------+-------------+--------+--------------------+-------------+--------------------+-----------+--------------------+----+------------------+-----------------+-------------------+------------+--------------+--------------------+---------------------+----+-----+----+
|Invoice/Item Number|      Date|Store Number|          Store Name|             Address|           City|Zip Code|      Store Location|County Number|       County|Category|       Category Name|Vendor Number|         Vendor Name|Item Number|    Item Description|Pack|Bottle Volume (ml)|State Bottle Cost|State Bottle Retail|Bottles Sold|Sale (Dollars)|Volume Sold (Liters)|Volume Sold (Gallons)|year|month|week|
+-------------------+----------+------------+--------------------+--------------------+---------------+--------+--------------------+-------------+-------------+--------+------------

In [28]:
#liquor_sales_df.show()

+-------------------+----------+------------+--------------------+--------------------+---------------+--------+--------------------+-------------+-------------+--------+--------------------+-------------+--------------------+-----------+--------------------+----+------------------+-----------------+-------------------+------------+--------------+--------------------+---------------------+
|Invoice/Item Number|      Date|Store Number|          Store Name|             Address|           City|Zip Code|      Store Location|County Number|       County|Category|       Category Name|Vendor Number|         Vendor Name|Item Number|    Item Description|Pack|Bottle Volume (ml)|State Bottle Cost|State Bottle Retail|Bottles Sold|Sale (Dollars)|Volume Sold (Liters)|Volume Sold (Gallons)|
+-------------------+----------+------------+--------------------+--------------------+---------------+--------+--------------------+-------------+-------------+--------+--------------------+-------------+---------

In [ ]:
liquor_date_df.describe()

In [11]:
from pyspark.sql.functions import when, isnan, count, col, monotonically_increasing_id, round

def find_row_missing_percentage(df):
    """
    This function takes a PySpark DataFrame as input and returns a dataframe after adding 
    columns that have missing values and the percentage of missing values in each row.

    :param df: PySpark DataFrame
    :return: PySpark DataFrame with percentage missing in each row
             List containing percentage of missing rows
    """
    # Count the number of columns
    total_columns = len(df.columns)

    # Find rows with missing values and count missing values in each row
    missing_values_df = df.select("*",
        sum(when(isnan(col(column_name)) | col(column_name).isNull(), 1).otherwise(0)
            for column_name in df.columns).alias("missing_count"))

    # Calculate the percentage of missing values for each row
    missing_percent_df = missing_values_df.withColumn("percent_missing", col("missing_count") / total_columns * 100)

    missing_percent_df = missing_percent_df.withColumn("percent_missing", round(missing_percent_df["percent_missing"], 0))

    # Calculate percentage of missing rows in dataframe 
    No_of_missRows = missing_percent_df.select('missing_count').where(missing_percent_df.missing_count>0).count()
    Percent_miss_rows = No_of_missRows/df.count()

    return missing_percent_df, Percent_miss_rows

In [2]:
liquor_date_df.describe()

DataFrame[summary: string, Invoice/Item Number: string, Store Number: string, Store Name: string, Address: string, City: string, Zip Code: string, Store Location: string, County Number: string, County: string, Category: string, Category Name: string, Vendor Number: string, Vendor Name: string, Item Number: string, Item Description: string, Pack: string, Bottle Volume (ml): string, State Bottle Cost: string, State Bottle Retail: string, Bottles Sold: string, Sale (Dollars): string, Volume Sold (Liters): string, Volume Sold (Gallons): string, year: string, month: string, week: string, lat: string, lon: string]

In [6]:
liquor_date_df.select("Date","week").show()

+----------+----+
|      Date|week|
+----------+----+
|2015-02-19|   8|
|2013-10-10|  41|
|2014-06-03|  23|
|2015-01-06|   2|
|2012-12-27|  52|
|2013-10-09|  41|
|2015-04-21|  17|
|2015-05-21|  21|
|2015-06-15|  25|
|2013-04-11|  15|
|2013-08-21|  34|
|2013-09-25|  39|
|2015-10-27|  44|
|2013-11-20|  47|
|2015-08-10|  33|
|2013-04-17|  16|
|2015-11-10|  46|
|2013-01-15|   3|
|2012-10-01|  40|
|2013-05-16|  20|
+----------+----+
only showing top 20 rows



In [10]:
liquor_date_df=liquor_date_df.withColumn("geometry_cleaned", regexp_replace("Store Location", "POINT \\(", ""))\
.withColumn("geometry_cleaned", regexp_replace("geometry_cleaned", "\\)", ""))\
.withColumn("lat", split(col("geometry_cleaned"), " ")[1].cast("float")) \
.withColumn("lon", split(col("geometry_cleaned"), " ")[0].cast("float"))\
.drop("geometry_cleaned").show()

+---------+----------+
|      lat|       lon|
+---------+----------+
|41.570843| -93.59701|
|  41.5541|-93.596756|
|40.761738| -95.38511|
| 41.76099| -93.57246|
|41.897053|-92.277756|
|42.517857| -94.53453|
| 43.14521| -95.14774|
|41.918327|  -91.4104|
|40.814667| -91.13666|
| 41.58498|-93.790535|
|41.009342|-92.437225|
| 43.26154| -93.63331|
|43.066994| -92.67556|
| 42.54299|-94.678055|
|41.261925| -95.87966|
|41.296227|-92.648155|
| 41.24394| -95.81877|
| 42.74173| -93.21693|
|41.575928| -93.61347|
|41.510082|-90.621544|
+---------+----------+
only showing top 20 rows



In [8]:
liquor_date_df=liquor_sales_df.withColumn('year', split(liquor_sales_df['Date'], '/').getItem(2)) \
   .withColumn('month', split(liquor_sales_df['Date'], '/').getItem(0)) 

In [9]:
liquor_date_df.select("year","month").show(1)

+----+-----+
|year|month|
+----+-----+
|2015|   02|
+----+-----+
only showing top 1 row



In [ ]:

liquor_sales_df = liquor_sales_df.withColumn("orderDateOnly", to_date("Order Date"))
liquor_sales_df=liquor_sales_df.withColumn("week", weekofyear("Order Date"))

In [3]:
df_date.count()

186850

In [22]:
##Electronic sales daily,weekly monthly,yearly

# df_date.groupBy("orderDateOnly", "Product") \
#     .agg(round(sum(col("Quantity Ordered") * col("Price Each")), 2).alias("salesAmount")) \
#     .show()



# df_date.groupBy("month", "Product") \
#     .agg(round(sum(col("Quantity Ordered") * col("Price Each")), 2).alias("salesAmount")) \
#     .show()


# df_date.groupBy("year", "Product") \
#     .agg(round(sum(col("Quantity Ordered") * col("Price Each")), 2).alias("salesAmount")) \
#     .show()


# df_date.groupBy("week", "Product") \
# .agg(round(sum(col("Quantity Ordered") * col("Price Each")), 2).alias("salesAmount")) \
#     .show()


# top_products = df_date.groupBy('Product') \
#                  .agg(round(sum(col("Quantity Ordered") * col("Price Each")), 2).alias('total_sales')) \
#                  .orderBy(desc('total_sales')) \
#                  .limit(10)

# top_products.show()

In [2]:
# # split() with withColumn
# df_date=df_sales_split.withColumn('year', split(df_sales_split['Order Date'], '-').getItem(0)) \
#        .withColumn('month', split(df_sales_split['Order Date'], '-').getItem(1)) 
# df_date = df_date.withColumn("orderDateOnly", to_date("Order Date"))

# df_date.show(truncate=False)

In [101]:
split_df = df_sales.withColumn("city", split(df_sales["Purchase Address"],",").getItem(1))\
.withColumn("state_code", split(df_sales["Order Date Address"],",").getItem(2))


In [114]:
df_sales_split.show()

+--------+--------------------+----------------+----------+--------------+--------------------+--------------+-----+
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|          city|state|
+--------+--------------------+----------------+----------+--------------+--------------------+--------------+-----+
|  141234|              iPhone|               1|     700.0|01/22/19 21:25|944 Walnut St, Bo...|        Boston|   MA|
|  141235|Lightning Chargin...|               1|     14.95|01/28/19 14:15|185 Maple St, Por...|      Portland|   OR|
|  141236|    Wired Headphones|               2|     11.99|01/17/19 13:33|538 Adams St, San...| San Francisco|   CA|
|  141237|    27in FHD Monitor|               1|    149.99|01/05/19 20:33|738 10th St, Los ...|   Los Angeles|   CA|
|  141238|    Wired Headphones|               1|     11.99|01/25/19 11:59|387 10th St, Aust...|        Austin|   TX|
|  141239|AAA Batteries (4-...|               1|      2.99|01/29

In [115]:
df_sales_split.withColumn("Order Date", to_date(unix_timestamp("Order Date", "MM/dd/yy HH:mm").cast("timestamp"))).show()

+--------+--------------------+----------------+----------+----------+--------------------+--------------+-----+
|Order ID|             Product|Quantity Ordered|Price Each|Order Date|    Purchase Address|          city|state|
+--------+--------------------+----------------+----------+----------+--------------------+--------------+-----+
|  141234|              iPhone|               1|     700.0|2019-01-22|944 Walnut St, Bo...|        Boston|   MA|
|  141235|Lightning Chargin...|               1|     14.95|2019-01-28|185 Maple St, Por...|      Portland|   OR|
|  141236|    Wired Headphones|               2|     11.99|2019-01-17|538 Adams St, San...| San Francisco|   CA|
|  141237|    27in FHD Monitor|               1|    149.99|2019-01-05|738 10th St, Los ...|   Los Angeles|   CA|
|  141238|    Wired Headphones|               1|     11.99|2019-01-25|387 10th St, Aust...|        Austin|   TX|
|  141239|AAA Batteries (4-...|               1|      2.99|2019-01-29|775 Willow St, Sa...| San 

In [105]:
df_sales_split.show(1)

+--------+-------+----------------+----------+--------------+--------------------+-------+-----+
|Order ID|Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|   city|state|
+--------+-------+----------------+----------+--------------+--------------------+-------+-----+
|  141234| iPhone|               1|     700.0|01/22/19 21:25|944 Walnut St, Bo...| Boston|   MA|
+--------+-------+----------------+----------+--------------+--------------------+-------+-----+
only showing top 1 row



In [ ]:
# df1 = df.withColumn('year', split(df['dob'], '-').getItem(0)) \
#        .withColumn('month', split(df['dob'], '-').getItem(1)) \
#        .withColumn('day', split(df['dob'], '-').getItem(2))
# df1.show(truncate=False)

In [81]:
split_df.show(2)

+--------+--------------------+----------------+----------+--------------+--------------------+--------------------+---------+---------+
|Order ID|             Product|Quantity Ordered|Price Each|    Order Date|    Purchase Address|        split_column|     city|    state|
+--------+--------------------+----------------+----------+--------------+--------------------+--------------------+---------+---------+
|  141234|              iPhone|               1|     700.0|01/22/19 21:25|944 Walnut St, Bo...|[944 Walnut St,  ...|   Boston| MA 02215|
|  141235|Lightning Chargin...|               1|     14.95|01/28/19 14:15|185 Maple St, Por...|[185 Maple St,  P...| Portland| OR 97035|
+--------+--------------------+----------------+----------+--------------+--------------------+--------------------+---------+---------+
only showing top 2 rows



In [74]:
df_sales1=df_sales.select(split(df_sales["Purchase Address"], ",").alias("city")[1])

In [75]:
df_sales1

DataFrame[split(Purchase Address, ,, -1) AS city[1]: string]

In [71]:
df_sales.select(split(df_sales["Purchase Address"], ",").alias("split_columns")[2]).show(1)

+--------------------------------------------------+
|split(Purchase Address, ,, -1) AS split_columns[2]|
+--------------------------------------------------+
|                                          MA 02215|
+--------------------------------------------------+
only showing top 1 row



In [62]:


books_ratings_df = books_ratings_df.withColumn("review/time", from_unixtime("review/time"))

In [60]:
books_ratings_df.show(1)

+----------+--------------------+-----+-------------+--------------------+------------------+------------+-------------------+--------------------+--------------------+
|        Id|               Title|Price|      User_id|         profileName|review/helpfulness|review/score|        review/time|      review/summary|         review/text|
+----------+--------------------+-----+-------------+--------------------+------------------+------------+-------------------+--------------------+--------------------+
|1882931173|Its Only Art If I...| null|AVCGYZL8FQQTD|"Jim of Oz ""jim-...|               7/7|         4.0|1999-10-23 08:00:00|Nice collection o...|This is only for ...|
+----------+--------------------+-----+-------------+--------------------+------------------+------------+-------------------+--------------------+--------------------+
only showing top 1 row



In [22]:



books_df.join(books_ratings_df,books_df.Title ==  books_ratings_df.Title,"inner").drop(books_ratings_df.Title).columns

['Title',
 'description',
 'authors',
 'image',
 'previewLink',
 'publisher',
 'publishedDate',
 'infoLink',
 'categories',
 'ratingsCount',
 'Id',
 'Price',
 'User_id',
 'profileName',
 'review/helpfulness',
 'review/score',
 'review/time',
 'review/summary',
 'review/text']

In [55]:
books_df_joined.count()

2999829

In [1]:
#books_df_joined.printSchema()

In [51]:
books_df.count()#212404
books_ratings_df.count()#3000000

212404

In [34]:
liquor_sales_df.printSchema()

root
 |-- Invoice/Item Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Store Number: integer (nullable = true)
 |-- Store Name: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zip Code: string (nullable = true)
 |-- Store Location: string (nullable = true)
 |-- County Number: integer (nullable = true)
 |-- County: string (nullable = true)
 |-- Category: integer (nullable = true)
 |-- Category Name: string (nullable = true)
 |-- Vendor Number: integer (nullable = true)
 |-- Vendor Name: string (nullable = true)
 |-- Item Number: string (nullable = true)
 |-- Item Description: string (nullable = true)
 |-- Pack: integer (nullable = true)
 |-- Bottle Volume (ml): integer (nullable = true)
 |-- State Bottle Cost: double (nullable = true)
 |-- State Bottle Retail: double (nullable = true)
 |-- Bottles Sold: integer (nullable = true)
 |-- Sale (Dollars): double (nullable = true)
 |-- Volume Sold (Liters): doub

In [14]:
category_mapping = {
    "1": "Smartphone",
    "2": "Charging Cable",
    "3": "Headphones",
    "4": "Monitor",
    "5": "Batteries",
    "6": "Laptop",
    "7": "TV",
    "8": "Dryer",
    "9": "Washing Machine",
    "10": "others"
}

In [5]:
category_mapping

{1: 'Smartphone',
 2: 'Charging Cable',
 3: 'Headphones',
 4: 'Monitor',
 5: 'Batteries',
 6: 'Laptop',
 7: 'TV',
 8: 'Dryer',
 9: 'Washing Machine',
 10: 'others'}

In [58]:
# when_conditions = []
# for category_id, category_name in category_mapping.items():
#     when_conditions.append(when(col("product").contains(category_name), category_name))

# mapped_df = df_date.withColumn("category", 
#                                   when(*when_conditions)
#                                   .otherwise("Others"))

In [61]:
df_date.withColumn("category",
                               when(col("product").contains("Batteries"), "Batteries")
                               .when(col("product").contains("Charging Cable"), "Charging Cable")
                               .when(col("product").contains("Phone"), "Smartphone")
                               .when(col("product").contains("Headphones"), "Headphones")
                               .when(col("product").contains("Laptop"), "Laptop")
                               .when(col("product").contains("TV"), "TV")
                               .when(col("product").contains("Dryer"), "Dryer")
                               .when(col("product").contains("Washing Machine"), "Washing Machine")
                               .otherwise("Others")).select("category","product").show()

+--------------+--------------------+
|      category|             product|
+--------------+--------------------+
|    Smartphone|              iPhone|
|Charging Cable|Lightning Chargin...|
|    Headphones|    Wired Headphones|
|        Others|    27in FHD Monitor|
|    Headphones|    Wired Headphones|
|     Batteries|AAA Batteries (4-...|
|        Others|27in 4K Gaming Mo...|
|Charging Cable|USB-C Charging Cable|
|    Headphones|Bose SoundSport H...|
|    Headphones|Apple Airpods Hea...|
|    Headphones|Apple Airpods Hea...|
|        Laptop|  Macbook Pro Laptop|
|     Batteries|AAA Batteries (4-...|
|        Others|    27in FHD Monitor|
|            TV|       Flatscreen TV|
|        Others|    27in FHD Monitor|
|    Smartphone|     Vareebadd Phone|
|    Headphones|Apple Airpods Hea...|
|Charging Cable|USB-C Charging Cable|
|     Batteries|AA Batteries (4-p...|
+--------------+--------------------+
only showing top 20 rows



In [51]:
books_df_joined.limit(10).select(explode(split(col("categories"), ", ")).alias("word")) \
    .groupBy("word") \
    .count() \
    .orderBy("count", ascending=False).show()

+--------------------+-----+
|                word|count|
+--------------------+-----+
|http://books.goog...|    3|
|                    |    2|
| a service for wh...|    2|
|http://books.goog...|    2|
|http://books.goog...|    1|
+--------------------+-----+



In [55]:
books_df_joined.select("categories").distinct()

28362

In [19]:
#df_date.select("Product").distinct().collect()

In [38]:
# sales_jan_df= read_in_data(spark, 'Sales_January_2019.csv')
# sales_feb_df= read_in_data(spark, 'Sales_February_2019.csv')
# sales_mar_df= read_in_data(spark, 'Sales_March_2019.csv')
# sales_april_df= read_in_data(spark, 'Sales_April_2019.csv')
# sales_may_df= read_in_data(spark, 'Sales_May_2019.csv')
# sales_june_df= read_in_data(spark, 'Sales_June_2019.csv')
# sales_july_df= read_in_data(spark, 'Sales_July_2019.csv')
# sales_aug_df= read_in_data(spark, 'Sales_August_2019.csv')
# sales_sep_df= read_in_data(spark, 'Sales_September_2019.csv')
# sales_oct_df= read_in_data(spark, 'Sales_October_2019.csv')
# sales_nov_df= read_in_data(spark, 'Sales_November_2019.csv')
# sales_dec_df= read_in_data(spark, 'Sales_December_2019.csv')


# def unionAll(*dfs):
#     return reduce(DataFrame.unionAll, dfs) 

# df_sales=unionAll(*[sales_jan_df, sales_feb_df,sales_mar_df,sales_april_df,sales_may_df,sales_june_df
#            ,sales_july_df,sales_aug_df,sales_sep_df,sales_oct_df,sales_nov_df,sales_dec_df])

In [23]:
#df_sales.describe()
#df_sales.select('Order Date').distinct().collect()
#df_sales.show().limit(3)

DataFrame[summary: string, Order ID: string, Product: string, Quantity Ordered: string, Price Each: string, Order Date: string, Purchase Address: string]

In [ ]:
#liquor_sales_df

In [ ]:
SERVER_ADDR_port = "income-mysql-db.cjuyq4k026ji.us-east-2.rds.amazonaws.com:3306"
db_name = "income"
f"jdbc:mysql://{SERVER_ADDR_port}/{db_name}"

In [41]:
books_ratings_df.limit(10).write \
  .format("jdbc") \
  .option("driver","com.mysql.cj.jdbc.Driver") \
  .mode("overwrite") \
  .option("url", f"jdbc:mysql://{SERVER_ADDR_port}/{db_name}") \
  .option("dbtable", "books_rating") \
  .option("user", "admin") \
  .option("password", "Logindatabase24") \
  .save()

In [40]:
liquor_sales_df.limit(10).write \
  .format("jdbc") \
  .option("driver","com.mysql.cj.jdbc.Driver") \
  .mode("overwrite") \
  .option("url", f"jdbc:mysql://{SERVER_ADDR_port}/{db_name}") \
  .option("dbtable", "liquor_sales") \
  .option("user", "admin") \
  .option("password", "Logindatabase24") \
  .save()

In [36]:
df_sales.limit(10).write \
  .format("jdbc") \
  .option("driver","com.mysql.cj.jdbc.Driver") \
  .mode("overwrite") \
  .option("url", f"jdbc:mysql://{SERVER_ADDR_port}/{db_name}") \
  .option("dbtable", "sales_data") \
  .option("user", "admin") \
  .option("password", "Logindatabase24") \
  .save()

In [5]:
SERVER_ADDR_port = "income-mysql-db.cjuyq4k026ji.us-east-2.rds.amazonaws.com:3306"
db_name = "income"

books_df.limit(100).write \
  .format("jdbc") \
  .option("driver","com.mysql.cj.jdbc.Driver") \
  .mode("overwrite") \
  .option("url", f"jdbc:mysql://{SERVER_ADDR_port}/{db_name}") \
  .option("dbtable", "books_data") \
  .option("user", "admin") \
  .option("password", "Logindatabase24") \
  .save()

spark.stop()

In [49]:
##Electronics data mart
SERVER_ADDR_port = "income-mysql-db.cjuyq4k026ji.us-east-2.rds.amazonaws.com:3306"
db_name = "income"

df_date_electronics.limit(1000).write \
  .format("jdbc") \
  .option("driver","com.mysql.cj.jdbc.Driver") \
  .mode("overwrite") \
  .option("url", f"jdbc:mysql://{SERVER_ADDR_port}/{db_name}") \
  .option("dbtable", "electronicsDataMart") \
  .option("user", "admin") \
  .option("password", "Logindatabase24") \
  .save()



In [8]:
##liquor data mart
SERVER_ADDR_port = "income-mysql-db.cjuyq4k026ji.us-east-2.rds.amazonaws.com:3306"
db_name = "income"

liquor_date_df.limit(30).write \
  .format("jdbc") \
  .option("driver","com.mysql.cj.jdbc.Driver") \
  .mode("overwrite") \
  .option("url", f"jdbc:mysql://{SERVER_ADDR_port}/{db_name}") \
  .option("dbtable", "liquorDataMart") \
  .option("user", "admin") \
  .option("password", "Logindatabase24") \
  .save()


In [3]:
books_df_joined.count()

2999829

In [20]:
books_df_joined.columns

['Title',
 'description',
 'authors',
 'image',
 'previewLink',
 'publisher',
 'publishedDate',
 'infoLink',
 'categories',
 'ratingsCount',
 'Id',
 'Title',
 'Price',
 'User_id',
 'profileName',
 'review/helpfulness',
 'review/score',
 'review/time',
 'review/summary',
 'review/text']

In [26]:
##Books data mart
SERVER_ADDR_port = "income-mysql-db.cjuyq4k026ji.us-east-2.rds.amazonaws.com:3306"
db_name = "income"

books_df_joined.limit(100).write \
  .format("jdbc") \
  .option("driver","com.mysql.cj.jdbc.Driver") \
  .mode("overwrite") \
  .option("url", f"jdbc:mysql://{SERVER_ADDR_port}/{db_name}") \
  .option("dbtable", "books_data_mart") \
  .option("user", "admin") \
  .option("password", "Logindatabase24") \
  .save()
#spark.stop()

In [ ]:
#   .option("fetchSize", 50000)\
#   .option("isolationLevel", "NONE")\
#   .option("queryTimeout", 600)\
 # .option("numPartitions", 10)\

In [2]:
print()

## handling nulls

In [40]:
df_temp = books_df_joined.limit(10)

replacements = {
    "int": 0,
    "double": 0.0,
    "float": 0.0,
    "string": ""
}

# Loop through columns and apply replacement rules
for col_name, data_type in df_temp.dtypes:
    replacement_value = replacements.get(data_type.lower(), None)
    if replacement_value is not None:
        df_temp = df_temp.withColumn(col_name, when(col(col_name).isNull(), replacement_value).otherwise(col(col_name)))

df_temp.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-----+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+
|         description|             authors|               image|         previewLink|           publisher|       publishedDate|            infoLink|          categories|        ratingsCount|        Id|Price|             User_id|         profileName|  review/helpfulness|        review/score|        review/time|      review/summary|         review/text|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+-----+--------------------+--------------------+--------------------+--------------------+-

In [56]:
books_df_joined.columns

['description',
 'authors',
 'image',
 'previewLink',
 'publisher',
 'publishedDate',
 'infoLink',
 'categories',
 'ratingsCount',
 'Id',
 'Price',
 'User_id',
 'profileName',
 'review/helpfulness',
 'review/score',
 'review/time',
 'review/summary',
 'review/text']

In [25]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, Word2Vec, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

# Preprocess the text data
preprocessed_df = df_temp.withColumn("categories_processed", lower(col("categories")))

# Tokenize the preprocessed text
tokenizer = Tokenizer(inputCol="categories_processed", outputCol="tokens")
tokenized_df = tokenizer.transform(preprocessed_df)

# Vectorize the tokens using Word2Vec
word2vec = Word2Vec(vectorSize=100, minCount=5, inputCol="tokens", outputCol="features")
model = word2vec.fit(tokenized_df)
vectorized_df = model.transform(tokenized_df)

# Convert string labels to numeric indices
label_indexer = StringIndexer(inputCol="categories", outputCol="label")
indexed_df = label_indexer.fit(vectorized_df).transform(vectorized_df)

# Train a classification model
train_df = indexed_df.filter(indexed_df["label"].isNotNull())
lr = LogisticRegression(featuresCol="features", labelCol="label", predictionCol="predicted_label")
pipeline = Pipeline(stages=[lr])
model = pipeline.fit(train_df)

# Impute missing categories
predict_df = indexed_df.filter(indexed_df["label"].isNull())
imputed_df = model.transform(predict_df)



In [17]:
imputed_df.write.csv("test_nlp.csv", header=True, mode="overwrite")

AnalysisException: CSV data source does not support array<string> data type.

In [5]:
df_temp = books_df_joined.limit(100)

from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, Word2Vec
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
# Preprocess the text data
# Assuming "categories" is the column containing text data
preprocessed_df = df_temp.withColumn("categories_processed", lower(col("categories")))

# Tokenize the preprocessed text
tokenizer = Tokenizer(inputCol="categories_processed", outputCol="tokens")
tokenized_df = tokenizer.transform(preprocessed_df)

# Vectorize the tokens using Word2Vec
word2vec = Word2Vec(vectorSize=100, minCount=5, inputCol="tokens", outputCol="features")
model = word2vec.fit(tokenized_df)
vectorized_df = model.transform(tokenized_df)

# Train a classification model
# Assuming there are some labels for training
train_df = vectorized_df.filter(vectorized_df["categories"].isNotNull())
lr = LogisticRegression(featuresCol="features", labelCol="categories", predictionCol="predicted_categories")
pipeline = Pipeline(stages=[lr])
model = pipeline.fit(train_df)

# Impute missing categories
predict_df = vectorized_df.filter(vectorized_df["categories"].isNull())
imputed_df = model.transform(predict_df)

# Select necessary columns and save the DataFrame with imputed categories
result_df = imputed_df.select("book_id", "title", "author", "categories", "predicted_categories")


IllegalArgumentException: requirement failed: Column categories must be of type numeric but was actually of type string.

In [9]:
train_df.select("categories").show()

+--------------------+
|          categories|
+--------------------+
|                    |
|http://books.goog...|
|http://books.goog...|
|http://books.goog...|
|                    |
| a service for wh...|
| a service for wh...|
|http://books.goog...|
|http://books.goog...|
|http://books.goog...|
|http://books.goog...|
|http://books.goog...|
|http://books.goog...|
|http://books.goog...|
|http://books.goog...|
|http://books.goog...|
|         ['Indiana']|
|         ['Indiana']|
|['Business & Econ...|
|['Business & Econ...|
+--------------------+
only showing top 20 rows

